In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import chi2_contingency
from pandas.plotting import scatter_matrix
import scipy.stats as stats
from sklearn.decomposition import PCA

In [2]:
df = pd.read_csv('heart.csv')
df

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


In [3]:
df_copy= df.copy()
df_copy = pd.get_dummies(df_copy, columns=['Sex'], drop_first=True)
df_copy = pd.get_dummies(df_copy, columns=['ChestPainType'], drop_first=True)
df_copy = pd.get_dummies(df_copy, columns=['ExerciseAngina'], drop_first=True)
df_copy = pd.get_dummies(df_copy, columns=['ST_Slope'], drop_first=True)
df_copy = pd.get_dummies(df_copy, columns=['RestingECG'], drop_first=True)
df_copy

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease,Sex_M,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,ExerciseAngina_Y,ST_Slope_Flat,ST_Slope_Up,RestingECG_Normal,RestingECG_ST
0,40,140,289,0,172,0.0,0,1,1,0,0,0,0,1,1,0
1,49,160,180,0,156,1.0,1,0,0,1,0,0,1,0,1,0
2,37,130,283,0,98,0.0,0,1,1,0,0,0,0,1,0,1
3,48,138,214,0,108,1.5,1,0,0,0,0,1,1,0,1,0
4,54,150,195,0,122,0.0,0,1,0,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,110,264,0,132,1.2,1,1,0,0,1,0,1,0,1,0
914,68,144,193,1,141,3.4,1,1,0,0,0,0,1,0,1,0
915,57,130,131,0,115,1.2,1,1,0,0,0,1,1,0,1,0
916,57,130,236,0,174,0.0,1,0,1,0,0,0,1,0,0,0


In [31]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid to search
param_grid = {
    'C': [0.1, 1, 10, 20, 300,1000],  # Regularization parameter
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],  # Kernel types
    'gamma': ['scale', 'auto'],  # Kernel coefficient for rbf/poly/sigmoid
    'degree': [1,2, 3, 4, 5, 6, 7]  # Degree of polynomial kernel
}

# Create the SVC model
svm_model = SVC()

# Instantiate the GridSearchCV object
grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy')

# Perform grid search
grid_search.fit(X_train_scaled, y_train)

# Best parameters found
print("Best parameters:", grid_search.best_params_)

# Predictions with best model
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_scaled)

# Evaluating the model
print("Classification report:")
print(classification_report(y_test, y_pred))


Best parameters: {'C': 1, 'degree': 1, 'gamma': 'scale', 'kernel': 'poly'}
Classification report:
              precision    recall  f1-score   support

           0       0.81      0.87      0.84        77
           1       0.90      0.85      0.88       107

    accuracy                           0.86       184
   macro avg       0.85      0.86      0.86       184
weighted avg       0.86      0.86      0.86       184



In [35]:
from sklearn.metrics import accuracy_score

# Assuming your data is stored in a pandas DataFrame called 'data'

# Splitting features and target variable
X = df_copy.drop('HeartDisease', axis=1)
y = df_copy['HeartDisease']

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Training the SVM model
svm_model = SVC(kernel='poly', C=1, gamma='scale',degree=1)  # You can change kernel and hyperparameters
svm_model.fit(X_train_scaled, y_train)

# Predictions
y_pred = svm_model.predict(X_test_scaled)

# Calculating accuracy
accuracy = accuracy_score(y_test, y_pred)

# Printing accuracy
print("Accuracy:", accuracy)

# Evaluating the model
print(classification_report(y_test, y_pred))


Accuracy: 0.8586956521739131
              precision    recall  f1-score   support

           0       0.81      0.87      0.84        77
           1       0.90      0.85      0.88       107

    accuracy                           0.86       184
   macro avg       0.85      0.86      0.86       184
weighted avg       0.86      0.86      0.86       184



In [36]:
from sklearn.metrics import accuracy_score

# Assuming your data is stored in a pandas DataFrame called 'data'

# Splitting features and target variable
X = df_copy.drop('HeartDisease', axis=1)
y = df_copy['HeartDisease']

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Training the SVM model
svm_model = SVC(kernel='rbf', C=1, gamma='scale')  # You can change kernel and hyperparameters
svm_model.fit(X_train_scaled, y_train)

# Predictions
y_pred = svm_model.predict(X_test_scaled)

# Calculating accuracy
accuracy = accuracy_score(y_test, y_pred)

# Printing accuracy
print("Accuracy:", accuracy)

# Evaluating the model
print(classification_report(y_test, y_pred))


Accuracy: 0.875
              precision    recall  f1-score   support

           0       0.84      0.87      0.85        77
           1       0.90      0.88      0.89       107

    accuracy                           0.88       184
   macro avg       0.87      0.87      0.87       184
weighted avg       0.88      0.88      0.88       184



In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report


# Splitting features and target variable
X = df_copy.drop('HeartDisease', axis=1)
y = df_copy['HeartDisease']

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Training the SVM model
svm_model = SVC(kernel='rbf', C=10, gamma='auto',degree=2)  # You can change kernel and hyperparameters
svm_model.fit(X_train_scaled, y_train)

# Predictions
y_pred = svm_model.predict(X_test_scaled)

# Evaluating the model
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.80      0.88      0.84        77
           1       0.91      0.84      0.87       107

    accuracy                           0.86       184
   macro avg       0.85      0.86      0.86       184
weighted avg       0.86      0.86      0.86       184



In [37]:
import pandas as pd

# Assuming your target variable is stored in a pandas DataFrame called 'df' and the column name is 'target'
class_distribution = df['HeartDisease'].value_counts()

print("Class Distribution:")
print(class_distribution)


Class Distribution:
1    508
0    410
Name: HeartDisease, dtype: int64
